## Code to move images from subfolders to main folders

In [ ]:
import os
import shutil
def move_images(main_folder):
    for root,dirs,files in os.walk(main_folder):
        print(root,dirs,files)
        for file in files:
            source = os.path.join(root,file)
            dest = os.path.join(main_folder,file)
            shutil.move(source,dest) 

In [ ]:
#Add the path to input directory
main_folder = '/users/PAS2186/balasubramaniam29/BIWA1/'
move_images(main_folder)

## Performing inference using hierarchical model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

In [ ]:
#Threshold values can be modified
class_thresholds = {
    'class' :0.6,
    'amphibia': 0.1,
    'aves': 0.5,
    'mammalia': 0.8,
    'serpentes': 0.9
}

labels = {
    'class' : ['amphibia', 'aves', 'invertebrates', 'lacertilia', 'mammalia', 'serpentes', 'testudines'],
    'serpentes': ["Butler's Gartersnake", "Dekay's Brownsnake", 'Eastern Gartersnake', 'Eastern Hog-nosed snake', 'Eastern Massasauga', 'Eastern Milksnake', 'Eastern Racer Snake', 'Eastern Ribbonsnake', 'Gray Ratsnake', "Kirtland's Snake", 'Northern Watersnake', 'Plains Gartersnake', 'Red-bellied Snake', 'Smooth Greensnake'],
    'mammalia': ['American Mink', 'Brown Rat', 'Eastern Chipmunk', 'Eastern Cottontail', 'Long-tailed Weasel', 'Masked Shrew', 'Meadow Jumping Mouse', 'Meadow Vole', 'N. Short-tailed Shrew', 'Raccoon', 'Star-nosed mole', 'Striped Skunk', 'Virginia Opossum', 'White-footed Mouse', 'Woodchuck', 'Woodland Jumping Mouse'],
    'aves': ['Common Yellowthroat', 'Gray Catbird', 'Indigo Bunting', 'Northern House Wren', 'Song Sparrow', 'Sora'],
    'amphibia': ['American Bullfrog', 'American Toad', 'Green Frog', 'Northern Leopard Frog']
}

In [ ]:
hierarchical_models = {}
for label in labels:
    hierarchical_models[label] = load_model(f"Models/inceptionv3_{label}.h5")

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)
    return img_array

def copy_images(source_dir,image_path,label,confidence,thresold):
    if confidence<thresold:
        human_review_dir = os.path.join(source_dir,'human',label)
        os.makedirs(human_review_dir, exist_ok=True)
        human_review_path = os.path.join(human_review_dir, os.path.basename(image_path))
        shutil.copy(image_path,human_review_path)
    else:
        output_path = os.path.join(source_dir,label)
        os.makedirs(output_path, exist_ok=True)
        output_dir = os.path.join(output_path, os.path.basename(image_path))
        shutil.copy(image_path,output_dir)

def process_images(input_d,level,dir_name):
    input_dir = os.path.join(input_d,dir_name)
    image_paths = [os.path.join(input_dir, fname) for fname in os.listdir(input_dir)]
    images = np.vstack([load_and_preprocess_image(image_path) for image_path in image_paths])
    predictions = hierarchical_models[level].predict(images)
    for image_path, prediction in zip(image_paths, predictions):
        predicted_class_index = np.argmax(prediction)
        predicted_class_label = labels[level][predicted_class_index]
        confidence = prediction[predicted_class_index]
        copy_images(input_d,image_path,predicted_class_label,confidence,class_thresholds[level])
    if level=="class":
        for label in ['serpentes','mammalia','aves','amphibia']:
            class_dir = os.path.join(input_d, label)
            if os.path.exists(class_dir):
                process_images(input_d,label,label)
            

In [ ]:
input_dir = "/Users/Desktop/"
process_images(input_dir,'class',"test")